# Overview
## Team Members: Matt Godfrey & Mariano Rivera
In this notebook, we take files that were extracted from Kaggle.com and transform and load them into a SQL database.

## Extract
We found two datasets on Whole Foods and Trader Joes locations from Kaggle.com. We downloaded the csv files (traderjoes.csv, whole foods.csv) and read them in as data frames.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
tj_filepath = "DataFiles/traderjoes.csv"
wf_filepath = "DataFiles/wholefoods.csv"

In [3]:
tj_data = pd.read_csv(tj_filepath)
wf_data = pd.read_csv(wf_filepath)

## Whole Foods Transformation
Clean up the dataframes by splitting the address columns into separate ones (city, street, state, & zip). We then dropped the longitude, latitude, & address columns. Next, we split up the zipcode column to separate the state ids from zip codes. Lastly, we dropped any other extraneous data. 

In [5]:
wf_data[["street", "city", "state", "zip", "none"]] = wf_data["address"].str.split(",", expand=True)
wf_data

,long,lat,name,address,street,city,state,zip,none
0,-86.589911,34.712156,Huntsville,"2501 Memorial Pkwy SW, Huntsville, AL 35801",2501 Memorial Pkwy SW,Huntsville,AL 35801,None,None
1,-88.150973,30.677104,Mobile,"3968 Airport Blvd, Mobile, AL 36608",3968 Airport Blvd,Mobile,AL 36608,None,None
2,-86.752324,33.460088,Mountain Brook,"3100 Cahaba Village Plaza, Birmingham, AL 35243",3100 Cahaba Village Plaza,Birmingham,AL 35243,None,None
3,-86.172682,32.358596,Montgomery,"1450 Taylor Rd, Montgomery, AL 36117",1450 Taylor Rd,Montgomery,AL 36117,None,None
4,-94.146200,36.113600,Fayetteville,"3425 N College Ave, Fayetteville, AR 72703",3425 N College Ave,Fayetteville,AR 72703,None,None
...,...,...,...,...,...,...,...,...,...
500,-122.461261,48.743633,Bellingham Lakeway,"1030 Lakeway Dr, Bellingham, WA 98229",1030 Lakeway Dr,Bellingham,WA 98229,None,None
501,-89.444164,43.074879,Madison,"3313 University Ave, Madison, WI 53705",3313 University Ave,Madison,WI 53705,None,None
502,-88.051510,43.075442,Wauwatosa,"11100 W Burleigh St, Wauwatosa, WI 53222",11100 W Burleigh St,Wauwatosa,WI 53222,None,None
503,-87.883762,43.060324,Milwaukee,"2305 N Prospect Ave, Milwaukee, WI 53211",2305 N Prospect Ave,Milwaukee,WI 53211,None,None


In [6]:
wf_data = wf_data.drop(columns=["long", "lat", "none", "address", "zip"])
wf_data

,name,street,city,state
0,Huntsville,2501 Memorial Pkwy SW,Huntsville,AL 35801
1,Mobile,3968 Airport Blvd,Mobile,AL 36608
2,Mountain Brook,3100 Cahaba Village Plaza,Birmingham,AL 35243
3,Montgomery,1450 Taylor Rd,Montgomery,AL 36117
4,Fayetteville,3425 N College Ave,Fayetteville,AR 72703
...,...,...,...,...
500,Bellingham Lakeway,1030 Lakeway Dr,Bellingham,WA 98229
501,Madison,3313 University Ave,Madison,WI 53705
502,Wauwatosa,11100 W Burleigh St,Wauwatosa,WI 53222
503,Milwaukee,2305 N Prospect Ave,Milwaukee,WI 53211


In [7]:
temp_df = wf_data["state"].str.split(expand=True)
temp_df

,0,1,2
0,AL,35801,None
1,AL,36608,None
2,AL,35243,None
3,AL,36117,None
4,AR,72703,None
...,...,...,...
500,WA,98229,None
501,WI,53705,None
502,WI,53222,None
503,WI,53211,None


In [8]:
wf_data = wf_data.assign(state=temp_df[0], zip=temp_df[1])
wf_data

,name,street,city,state,zip
0,Huntsville,2501 Memorial Pkwy SW,Huntsville,AL,35801
1,Mobile,3968 Airport Blvd,Mobile,AL,36608
2,Mountain Brook,3100 Cahaba Village Plaza,Birmingham,AL,35243
3,Montgomery,1450 Taylor Rd,Montgomery,AL,36117
4,Fayetteville,3425 N College Ave,Fayetteville,AR,72703
...,...,...,...,...,...
500,Bellingham Lakeway,1030 Lakeway Dr,Bellingham,WA,98229
501,Madison,3313 University Ave,Madison,WI,53705
502,Wauwatosa,11100 W Burleigh St,Wauwatosa,WI,53222
503,Milwaukee,2305 N Prospect Ave,Milwaukee,WI,53211


In [18]:
wf_data = wf_data[wf_data["zip"].str.contains("-")==False]
wf_data = wf_data[wf_data["zip"].str.contains("\?")==False]
wf_data = wf_data[wf_data["zip"].str.contains("Clara")==False]
wf_data = wf_data[wf_data["city"].str.contains("Ste")==False]
wf_data = wf_data[wf_data["city"].str.contains("Suite")==False]
wf_data = wf_data[wf_data["zip"].str.contains("Beach")==False]
wf_data

,name,street,city,state,zip
0,Huntsville,2501 Memorial Pkwy SW,Huntsville,AL,35801
1,Mobile,3968 Airport Blvd,Mobile,AL,36608
2,Mountain Brook,3100 Cahaba Village Plaza,Birmingham,AL,35243
3,Montgomery,1450 Taylor Rd,Montgomery,AL,36117
4,Fayetteville,3425 N College Ave,Fayetteville,AR,72703
...,...,...,...,...,...
500,Bellingham Lakeway,1030 Lakeway Dr,Bellingham,WA,98229
501,Madison,3313 University Ave,Madison,WI,53705
502,Wauwatosa,11100 W Burleigh St,Wauwatosa,WI,53222
503,Milwaukee,2305 N Prospect Ave,Milwaukee,WI,53211


## Trader Joe's Transformation
Clean up the dataframes by dropping columns (landline, mobile, and website). Next, we dropped any other extraneous data. Lastly, renamed the name column. 

In [11]:
tj_data= tj_data.drop(columns=["landline", "mobile", "website"])
tj_data

,store_name,street,city,state,zip
0,Trader Joe's Rancho Palos Verdes (233),31176 Hawthorne Blvd,Rancho Palos Verdes,CA,90275
1,Trader Joe's Charlotte (744),1133 Metropolitan Ave,Charlotte,NC,28204
2,Trader Joe's Sunnyvale (68),316 West El Camino Real,Sunnyvale,CA,94087
3,Trader Joe's Corona (213),2790 Cabot Dr,Corona,CA,92883
4,Trader Joe's El Cerrito (108),225 El Cerrito Plaza,El Cerrito,CA,94530
...,...,...,...,...,...
516,Trader Joe's Chicago - Lincoln Park (691),1840 N Clybourn Ave,Chicago,IL,60614
517,Trader Joe's Concord (83),785 Oak Grove Rd,Concord,CA,94518
518,Trader Joe's San Diego - Point Loma (188),2401 Truxtun Rd,San Diego,CA,92106
519,Trader Joe's Sandy Springs (731),6277 Roswell Road NE,Sandy Springs,GA,30328


In [12]:
tj_data = tj_data[tj_data["zip"].str.contains("-")==False]
tj_data = tj_data[tj_data["zip"].str.contains("\?")==False]
tj_data

,store_name,street,city,state,zip
0,Trader Joe's Rancho Palos Verdes (233),31176 Hawthorne Blvd,Rancho Palos Verdes,CA,90275
1,Trader Joe's Charlotte (744),1133 Metropolitan Ave,Charlotte,NC,28204
2,Trader Joe's Sunnyvale (68),316 West El Camino Real,Sunnyvale,CA,94087
3,Trader Joe's Corona (213),2790 Cabot Dr,Corona,CA,92883
4,Trader Joe's El Cerrito (108),225 El Cerrito Plaza,El Cerrito,CA,94530
...,...,...,...,...,...
516,Trader Joe's Chicago - Lincoln Park (691),1840 N Clybourn Ave,Chicago,IL,60614
517,Trader Joe's Concord (83),785 Oak Grove Rd,Concord,CA,94518
518,Trader Joe's San Diego - Point Loma (188),2401 Truxtun Rd,San Diego,CA,92106
519,Trader Joe's Sandy Springs (731),6277 Roswell Road NE,Sandy Springs,GA,30328


In [13]:
tj_data_clean= tj_data.rename(columns={"store_name": "name"})
tj_data_clean

,name,street,city,state,zip
0,Trader Joe's Rancho Palos Verdes (233),31176 Hawthorne Blvd,Rancho Palos Verdes,CA,90275
1,Trader Joe's Charlotte (744),1133 Metropolitan Ave,Charlotte,NC,28204
2,Trader Joe's Sunnyvale (68),316 West El Camino Real,Sunnyvale,CA,94087
3,Trader Joe's Corona (213),2790 Cabot Dr,Corona,CA,92883
4,Trader Joe's El Cerrito (108),225 El Cerrito Plaza,El Cerrito,CA,94530
...,...,...,...,...,...
516,Trader Joe's Chicago - Lincoln Park (691),1840 N Clybourn Ave,Chicago,IL,60614
517,Trader Joe's Concord (83),785 Oak Grove Rd,Concord,CA,94518
518,Trader Joe's San Diego - Point Loma (188),2401 Truxtun Rd,San Diego,CA,92106
519,Trader Joe's Sandy Springs (731),6277 Roswell Road NE,Sandy Springs,GA,30328


## Load 
Set up the connection to the grocery database. Using that connection, we load the clean dataframes into the SQL database. 

In [14]:
connection_string = "postgres:bootcamp@localhost:5432/grocery_db"
engine = create_engine(f'postgresql://{connection_string}')

In [15]:
# Confirm tables
engine.table_names()

<ipython-input-15-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['whole_foods', 'trader_joes']

In [16]:
#Load the data
tj_data_clean.to_sql(name='trader_joes', con=engine, if_exists='append', index=False)

In [19]:
wf_data.to_sql(name='whole_foods', con=engine, if_exists='append', index=False)